In [2]:
# Homework 6 Template
# Jimena Stephenson 

In [1]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# my modules
from ReadFile import Read
from CenterOfMass2 import CenterOfMass



In [2]:
def OrbitCom(galaxy, start, end, n=5):
    """function that loops over all the desired snapshots to compute the COM pos and vel as a function of time.
    inputs:
    galaxy name ie: "MW", start: first snapshot to be read, end: last snapshot to be read, n: intervals to return the COM
    We will compute up to snapshot 800 and we will outpu values in intervals of n=5      
    returns: 
    computes the timen and COM position and velocity vectors of a given a galaxy in each snapshot
    saves that output into a file.
    """
    
    # compose the filename for output
    fileoutmw = "orbit_MW.txt"
    fileoutm31 = "orbit_M31.txt"
    fileoutm33 = "orbit_M33.txt"
    
    #  set tolerance and VolDec for calculating COM_P in CenterOfMass
    # for M33 that is stripped more, use different values for VolDec
    
    if (galaxy == 'M33'):
        delta = 0.1
        VolDec=4
        
    else:
        delta = 0.1
        VolDec = 2

    
    # generate the snapshot id sequence 
    # it is always a good idea to also check if the input is eligible (not required)
    snap_ids = np.arange(start, end+1, step=n)
    
    
    # initialize the array for orbital info: t, x, y, z, vx, vy, vz of COM
    
    orbit = np.zeros((snap_ids.size, 7))
    
    # a for loop 
    for  i, snap_id in enumerate(snap_ids):
        
        # compose the data filename (be careful about the folder)
         # inputs to recontruct the filenumber to the value "000"
        ilbl = '000' + str(snap_id)
        # remove all but the last 3 digits
        ilbl = ilbl[-3:]
        filename = "%s_"%(galaxy) + ilbl + '.txt'
        
        # Initialize an instance of CenterOfMass class, using disk particles
        
        COM = CenterOfMass(filename, 2)

        # Store the COM pos and vel. Remember that now COM_P required VolDec
        COM_P = COM.COM_P(delta, VolDec)
        COM_V = COM.COM_V(COM_P[0],COM_P[1],COM_P[2])

    
        # store the time, pos, vel in ith element of the orbit array,  without units (.value) 
        # note that you can store 
        # a[i] = var1, *tuple(array1)
        
        orbit[i] = COM.time.value/1000, *tuple(COM_P.value), *tuple(COM_V.value)
        
        # print snap_id to see the progress
        #print(snap_id)
        
    # write the data to a file
    # we do this because we don't want to have to repeat this process 
    # this code should only have to be called once per galaxy.
    #np.savetxt(fileoutmw, orbit, fmt = "%11.3f"*7, comments='#',
               #header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      #.format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
    return orbit

In [3]:
#OrbitCom('MW', 0, 10, n=5)

In [5]:
# Recover the orbits and generate the COM files for each galaxy
# read in 800 snapshots in intervals of n=5
# Note: This might take a little while - test your code with a smaller number of snapshots first!
fileoutmw = "Orbit_MW.txt"
fileoutm31 = "Orbit_M31.txt"
fileoutm33 = "Orbit_M33.txt"

OrbitMW = OrbitCom("MW",0,801,5)
np.savetxt(fileoutmw, OrbitMW, fmt = "%11.3f"*7, comments='#',
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))

OrbitM31 = OrbitCom("M31",0,801,5)
np.savetxt(fileoutm31, OrbitM31, fmt = "%11.3f"*7, comments='#',
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))

OrbitM33 = OrbitCom("M33",0,801,5)
np.savetxt(fileoutm33, OrbitM33, fmt = "%11.3f"*7, comments='#',
               header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                      .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))


In [8]:
# Read in the data files for the orbits of each galaxy that you just created
# headers:  t, x, y, z, vx, vy, vz
# using np.genfromtxt
OrbitMW_data = np.genfromtxt(fileoutmw,dtype=None,names=True)
OrbitM31_data = np.genfromtxt(fileoutm31,dtype=None,names=True)
OrbitM33_data = np.genfromtxt(fileoutm33,dtype=None,names=True)

In [9]:
# function to compute the magnitude of the difference between two vectors 
# You can use this function to return both the relative position and relative velocity for two 
# galaxies over the entire orbit  
MWx = OrbitMW_data['x']
MWy = OrbitMW_data['y']
MWz = OrbitMW_data['z']
MWvx = OrbitMW_data['vx']
MWvy = OrbitMW_data['vy']
MWvz = OrbitMW_data['vz']
    
M31x = OrbitM31_data['x']
M31y = OrbitM31_data['y']
M31z = OrbitM31_data['z']
M31vx = OrbitM31_data['vx']
M31vy = OrbitM31_data['vy']
M31vz = OrbitM31_data['vz']
    
M33x = OrbitM33_data['x']
M33y = OrbitM33_data['y']
M33z = OrbitM33_data['z']
M33vx = OrbitM33_data['vx']
M33vy = OrbitM33_data['vy']
M33vz = OrbitM33_data['vz']

def mag():
    
    mag_sep1 = np.zeros(np.size(MWx))
    mag_vel1 = np.zeros(np.size(MWvx))
    mag_sep2 = np.zeros(np.size(M33x))
    mag_vel2 = np.zeros(np.size(M33vx))
    
    for i in range(0,len(MW_x)):
        mwm31 = [MWx[i]-M31x[i], MWy[i]-M31y[i], MWz[i]-M31z[i]]
        mwm31_mag = np.sqrt(mwm31[0]**2+mwm31[1]**2+mwm31[2]**2)
        mag_mwm31[i] = mwm31_mag 
        
        vmwm31 = [MWvx[i]-M31vx[i], MWvy[i]-M31vy[i], MWvz[i]-M31vz[i]]
        vmwm31_mag = np.sqrt(vmwm31[0]**2+vmwm31[1]**2+vmwm31[2]**2)
        mag_vmwm31[i] = vmwm31_mag
        
        m31m33 = [M31x[i]-M33x[i], M31y[i]-M33y[i], M31z[i]-M33z[i]]
        m31m33_mag = np.sqrt(m31m33[0]**2+m31m33[1]**2+m31m33[2]**2)
        mag_m31m33[i] = m31m33_mag
        
        vm31m33 = [M31vx[i]-M33vx[i], M31vy[i]-M33vy[i], M31vz[i]-M33vz[i]]
        vm31m33_mag = np.sqrt(vm31m33[0]**2+vm31m33[1]**2+vm31m33[2]**2)
        mag_vvm31m33[i] = vm31m33_mag
    
    return mag_mwm31, mag_vmwm31, mag_m31m33, mag_vm31m33


In [ ]:
# Determine the magnitude of the relative position and velocities 

# of MW and M31

# of M33 and M31


In [ ]:
# Plot the Orbit of the galaxies 
#################################



In [ ]:
# Plot the orbital velocities of the galaxies 
#################################
